## Week 2-2: Ridge regression

#### Announcement
* HW due this Friday
* TA office hour this afternoon 3:00 - 5:00pm
* Final report due June 06 @ 11:59pm

#### Last time
* Subset selection methods

#### Today
* Ridge regression


#### References

- Belkin, M., D. Hsu, S. Ma, and S. Mandal (2019). Reconciling modern machine learning practice and the bias-variance trade-off. _Proceedings of the National Academy of Sciences_ 116:15849--15854.
- ESL, Chapter 3
- Hoerl, A. E. and Kennard, R. W. (1970). Ridge regression: biased estimation for non-orthogonal problems. _Technometrics_, 12:55--67.
- Mei, S. and A. Montanari (2020). The generalization error of random features regression: Precise asymptotics and double descent curve. https://arxiv.org/pdf/1908.05355.pdf
- Theobald, C. M. (1974). Generalizations of Mean Square Error Applied to Ridge Regression. _JRSSB_. 36:103--106
- Wainwright, W. (2019). High-dimensional statistics: A non-asymptotic viewpoint. _Cambridge Series in Statistical and Probabilitistic Mathematics_.
- van Wieringen, W. N. (2021). Lecture notes on ridge regression. [https://arxiv.org/pdf/1509.09169.pdf]

#### Recall subset selection 

The best subset selection solves

$$\min_{\beta \in \mathbb{R}^p} \left\{ \|Y - X\beta\|_2^2 + \lambda \|\beta\|_0 \right\}$$


Is $\|\beta \|_0$ a norm? 


Vector norm $\|\cdot\|: \mathbb{R}^n \to \mathbb{R}$, for $a \in \mathbb{R}^{n}$: 
- $\|a\| \geq 0$
- $\|a\| = 0$ if and only if $a = 0$
- homogeneity: $\|ca\| = c\|a\|$, $c \geq 0$
- triangle inequality: $\|a + b\| \leq \|a\| + \|b\|$


<img src="lq-ball.png" style="width: 600px;"/>

Plot of $\|a\|_q$-norm: (a) $q=1$, (b) $q=0.75$, (c) $q = 0.5$.

(Source: Chapter 7 of High-dimensional statistics by Martin J. Wainwright (2019))

#### Convex relaxation

* LASSO (least absolute shrinkage and selection operator) replaces $\|\beta\|_0$ with $\|\beta\|_1$. Solving

$$\min_{\beta \in \mathbb{R}^p} \left\{ \|Y - X\beta\|_2^2 + \lambda \|\beta\|_1 \right\}$$

* Ridge regression. Solving

$$\min_{\beta \in \mathbb{R}^p} \left\{ \|Y - X\beta\|_2^2 + \lambda \|\beta\|_2^2 \right\}$$

## Ridge regression 

<img src="ridge-regression.png" style="width: 900px;"/>

Solving $\min_{\beta \in \mathbb{R}^p} \left\{ \|Y - X\beta\|_2^2 + \lambda \|\beta\|_2^2 \right\}$, we obtain 
$$
\hat \beta_{\text{ridge}} = (X'X + \lambda I_p)^{-1} X'y
$$


$p < n$

- Colinearity

- Show $\hat \beta_{\text{ridge}}$ is biased 
- Derive the variance of $\hat \beta_{\text{ridge}}$



<img src="bias.png" style="width: 600px;"/>

Source: [Link](https://www.few.vu.nl/~wvanwie/Courses/HighdimensionalDataAnalysis/WNvanWieringen_HDDA_Lecture234_RidgeRegression_20182019.pdf)

<img src="variance.png" style="width: 400px;"/>

<img src="contour.png" style="width: 600px;"/>

<img src="MSE.png" style="width: 800px;"/>

#### Solving ridge regression using SVD

Let $X = UDV'$, the thin SVD for $X$,
where $U = (u_1, \dots, u_p)$, $V = (v_1, \dots, v_p)'$, and $D = \text{diag}(d_1, \dots, d_p)$

- $p < n$
- $X \in \mathbb{R}^{n \times p}$
- $U \in \mathbb{R}^{n \times p}$
- $D \in \mathbb{R}^{p \times p}$
- $V \in \mathbb{R}^{p \times p}$
- $u_k \in \mathbb{R}^{n}$
- $v_k \in \mathbb{R}^{p}$

Then the ridge solution is given by
$$
\hat \beta_{\text{ridge}} = \sum_{j=1}^p \frac{d_j v_ju_j'}{d_j^2 + \lambda} y
$$

What about when $p > n$? [Bad example](https://anujkatiyal.com/blog/2017/09/30/ml-regression/#.Yk5lc27MIRQ)

In python, you can use [this package](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) (solver for nonsparse matrix, use cholesky)


In [22]:
from sklearn.linear_model import Ridge
import numpy as np
import timeit


n_samples, n_features = 100, 100000
rng = np.random.RandomState(seed=2022)
y = rng.randn(n_samples)
X = rng.randn(n_samples, n_features)

start = timeit.default_timer() # start timer
clf = Ridge(alpha=0.2, solver = "auto")
clf.fit(X, y)
stop = timeit.default_timer() # stop timer
print('Time: ', stop - start)  
clf.coef_[0:10]

Time:  0.05127781299961498


array([ 2.33864885e-05, -1.04650148e-04,  3.37335092e-05,  2.15090793e-04,
        1.72066669e-04, -1.77999487e-04,  2.26323266e-05,  3.90972968e-05,
       -3.21042395e-06, -1.65162587e-04])

In [23]:
start = timeit.default_timer() # start timer
clf = Ridge(alpha=0.2, solver = "svd")
clf.fit(X, y)
stop = timeit.default_timer() # stop timer
print('Time: ', stop - start)  
clf.coef_[0:10]

Time:  1.0505208269969444


array([ 2.33864885e-05, -1.04650148e-04,  3.37335092e-05,  2.15090793e-04,
        1.72066669e-04, -1.77999487e-04,  2.26323266e-05,  3.90972968e-05,
       -3.21042395e-06, -1.65162587e-04])

In [25]:
start = timeit.default_timer() # start timer
clf = Ridge(alpha=0.2, solver = "cholesky")
clf.fit(X, y)
stop = timeit.default_timer() # stop timer
print('Time: ', stop - start)  
clf.coef_[0:10]

# I found that using the cholesky is the fastest

Time:  0.06843933499476407


array([ 2.33864885e-05, -1.04650148e-04,  3.37335092e-05,  2.15090793e-04,
        1.72066669e-04, -1.77999487e-04,  2.26323266e-05,  3.90972968e-05,
       -3.21042395e-06, -1.65162587e-04])

#### Degrees of freedom

In usual linear regression setting, the degrees of freedom of a model is the number of free parameters $p$. In ridge regression, we define the _effective degress of freedom_ as 
$$
df(\lambda) = \text{tr}[X'(X'X + \lambda I_p)^{-1} X'] = \sum_{j=1}^p \frac{d_j^2}{d_j^2 + \lambda},
$$
which is a monotone decreasing function of $\lambda$. 

- when $\lambda = 0$, the effective degrees of freedom is $p$;
- when $\lambda \to \infty$, the effective degress of freedom $\to 0$.

#### Connection to Bayesian method (optional, if time permits)

In many textbook or lecture notes, they want to connect ridge regression with Bayesian method. Here is why:

- Bayes rule: 

$$
\pi(\beta | X) = \frac{p(x|\beta) \pi(\beta)}{\int p(x|\beta) \pi(\beta) d\beta}
$$

- The likelihood function is $p(x | \beta) = N(y - X\beta, \sigma^2I_n)$
- Prior for $\beta$ is $\pi(\beta) = N(0, \tau^2 I_p)$
- Then one can show that (let $\sigma^2 = 1$ and $1/\tau^2 = \lambda$
$$
\pi(\beta | X) \sim N((X'X + \lambda I_p)^{-1} X'y, (X'X + \lambda I_p)^{-1})
$$
- The MAP (maximum a posteriori estimation) is the ridge regression 


### Ridge regression in high-dimensional (double descent phenomenon, active research area)

* When $p > n$, interesting thing happens. 


<img src="double-descent.png" style="width: 800px;"/>


Source: [website](https://m-clark.github.io/posts/2021-10-30-double-descent/)


* Mikhail Belkin, Daniel Hsu, Siyuan Ma, and Soumik Mandal (2019). Reconciling modern machine learning practice and the bias-variance trade-off. _Proceedings of the National Academy of Sciences_ 116:15849--15854.
* Song Mei and Andrea Montanari (2020). The generalization error of random features regression: Precise asymptotics and double descent curve. https://arxiv.org/pdf/1908.05355.pdf
* Mei's slides [here](https://www.stat.berkeley.edu/~songmei/Teaching/STAT260_Spring2021/Lecture_notes/Lecture_24_DD.pdf)